# Pisces trial data pre-processing and database upload

## This document describes the steps required to pre-process the Pisces trial data for upload

The ongoing Pisces trials for both M&S and Tesco projects use the same methodology, so it makes sense to store the data in the same database. Each cruise however has multiple data sources, and these need to be screened, processed, and standardized into a form that enables the data to be uploaded to a database for storage and to enable as painless an import procedure into an analysis program. In an ideal world, we would have machine-collected data that can be simply sucked into a data table as-is. In the real world, Sod's law prevails so we have to do quite a bit of wrangling to get the data into the right format.  


There are four different sources of data:  
1. **CatchApp**. Each vessel has CatchApp on an iPad, and this is our primary record of which tows were carried out on any given day. The CatchApp data are downloaded from www.succorfish.net. Some of the fields require manual entry of character field values, so unfortunately we have to screen these during pre-processing. But in general CatchApp is a great tool and vast improvement over handwritten sheets. CatchApp is split into two tables:
 - Vessel, date, Pisces status. This forms the basis of our core identifier table
 - Catch species and retained/discarded weights. This forms the catch table
2. **Followmee**. This is another great tool, and is a GPS tracker on the iPad. It pings approximately every 2-10 minutes, and from Followmee we get the start and end of each haul.
  - The track data is uploaded with only a few additions directly to the database. It has a date-time stamp, so is readily queryable.  
3. **Effort**. CatchApp provides a field for haul duration, but the estimate is not particularly reliable. Consequently, I record the gear-in/gear-out time and position manually by going through the tracks day by day, and work out from the vessel speed and course whether they are towing or not. Eventually I can train a model to predict this. It wouldn't be hard. When the vessel slows to ~ 3 kts it is towing. This is easier to do for the larger vessels. In the case of *Eilidh Anne* we can't tell whether she is fishing or not. At best we can see the start of the first haul of the day, and the end of the last haul.  
4. **Deck photos**. These will hopefully form the basis of a hopper catch calibration. Currently the skipper sends a photo of the catch to the project phone via WhatsApp. Unfortunately this does not contain the EXIF (image information) of when the photo was actually taken. Additionally, the photo filename is not consistent and changes with different downloads due to iOS nonsense. To deal with this, a copy of the photo is renamed to a format that identifies where and when the image was taken. The image is stored in a data directory, and the filename entered into a database table. This enables retrieval of the photo using the sample identification variable. Note: Photos are typically taken of the hopper, but sometimes additional photos of the catch sample are taken. This is taken care of in the file renaming step.  Of all the processes, this is probably the most manually intensive and error-prone. I have automated it as much as possible.  
5. **Catch subsample bulk and fish length**. These data are collected only when there is an observer on board. A subsample of the catch (one bucket) is collected, and different elements of the catch weighed and the fishes are measured (total length rounded down to the nearest centimeter, except for skates which are measured tip to tip). Currently these data are entered in a Google Sheet. The format itself is not conducive for direct read-in to the database. However, its current form is tractable for the person on the boat entering it, so it is quicker to leave that system in place and modify it later. The sequence is:
  - Restructure a copy of the data into individual columns (one per variable)
  - Read the individual sheets corresponding to an individual haul into R, making sure the haul sample identifiers are correctly specified
  - Split into fish length and sample weight database tables
   
```{note}
Sometimes we need to modify the database structure to accommodate idiosyncrasies in the data. For example, Eilidh Anne effort measurements are *per day*, with a fudge factor for net retrieval and re-deployment. This requires a separate database table. It is what it is...
```  

````{margin}
```{note}
The hard code samples (well, as much as they can be hard given I have to continually modify them) are written in Jupyter notebooks. As things settle in and I cover the range of idiosyncrasies, I will formalise these a bit more into a closed repository on Github.
```
````

````{margin}
```{note}
As we add vessels protocols may change.For example, in the first Golden Ray trial a paired design was tested. This requires a change to the trialID table. RDBMS are structured, but we can add a degree of flexibility where required.
```
````

The pre-processing steps take the five data sources and screen, standardize, and restructure to give eight csv files. These files are then read directly into the database using the generic sql statement

```
LOAD DATA INFILE csv 
IGNORE INTO tablename
```
There will be more of that later in the next section.  
This gives us the following schema (links are omitted for clarity, and because not all tables have primary keys):

```{figure} PiscesTrialDatabaseSchema.png
```

## Reading and splitting the CatchApp data
Each vessel has its own Jupyter Notebook in which I have to accumulate and deal with vesel-specific idiosyncrasies. What follows is a broad-brush treatment of the steps required.  

First step is to read in the raw CatchApp file and deal with specific issues like specifying which project the data should be assigned to, make a primary key sampleID, add a variable in (Year, Month, Day) format that MySQL can read as a date variable, tidu up some variable names, and synonymise any typos.  
To give you some idea of what's involved, this is the code from Virtuous after the first trip (click to show the code):

{
    "tags": [
        "hide_input",
    ]
}
```
infile <- './Data/VIRTUOUS-FR253-.csv'
df <- read.csv(infile, stringsAsFactors=F)

#Rename variables for consistency

df$Project <- 'M&S' 
df$Bait <- trimws(df$Bait)
df$Bait[df$Bait == 'No lights'] <- 'No light'
df$Bait[df$Bait == 'White light constant bright'] <- 'White, constant, bright'
df$Bait[df$Bait == 'White on bright smp'] <- 'White, constant, bright'

df$Position <- 'SMP'
df$Activity <- gsub(' ', '', df$Activity)
df$Asset[df$Asset == 'VIRTUOUS FR253'] <- 'Virtuous'


#Generate sample PK
df$SampleID <- paste(df$Asset, df$Start.Date, df$Activity, sep='_')

#Don't generate Date values for MYSQL
#df$Date_Start <- as.Date(df$Start.Date, format = "%d/%m/%Y")
#df$Date_End <- as.Date(df$End.Date, format = "%d/%m/%Y")

#Generate Lat-Longs
df <- cbind(df, strcapture("(.*),(.*)", as.character(df$Location), data.frame(Latitude = "", Longitude = "")))
df$Latitude <- as.numeric(df$Latitude)
df$Longitude <- as.numeric(df$Longitude)

df$Soak.tow.time <- df$Soak...tow.time 
df$Soak...tow.time <- NULL

df$Retained.Weight <- df$Retained.weight 
df$Retained.weight <- NULL

df$Returned.Reason <- df$Returned.reason 
df$Returned.reason <- NULL

df$Retained.Number <- df$Retained.No. 
df$Retained.No. <- NULL

df$Gear.No <- df$Gear.No. 
df$Gear.No. <- NULL


# MySQL reads year-month-day into dates
df$year <- substr(df$Start.Date, 7, 11)
df$month <- substr(df$Start.Date, 4, 5)
df$day <- substr(df$Start.Date, 1, 2)

df$YearMonthDay <- paste(df$year, df$month, df$day, sep='-')

df$year <- NULL
df$month <- NULL
df$day <- NULL

#Parse into constituent light elements

for (i in 1:nrow(df)) {
if (df[i,'Bait'] == 'No light'){
       df[i,'Light'] <- 'Off'
       df[i,'Colour'] <- 'None'
       df[i,'Flash'] <- 'None'
       df[i,'Intensity'] <- 'None'}

else if (df[i,'Bait'] == 'White, constant, bright'){
       df[i,'Light'] <- 'On'
       df[i,'Colour'] <- 'White'
       df[i,'Flash'] <- 'Constant'
       df[i,'Intensity'] <- 'Bright'
    }
    
}
```

Of course as more typos etc. accumulate, we need to constantly update the notebook to accommodate this.  
But once this is done, we can easily write two of the important tables: The trialID table, and the catchData table:

{
    "tags": [
        "hide_input",
    ]
}
```
trialID <- unique(df[,c("SampleID", "Project", "Asset", "YearMonthDay", "Start.Date", "End.Date", "Location", "Activity", 
"Gear", "Gear.Type", "Gear.No", "Bait", "Gear.info",
"Latitude", "Longitude", "Soak.tow.time", "Position", "Light", "Colour", 
"Flash", "Intensity")])
trialID <- trialID[order(trialID$SampleID),]
write.csv(trialID, file='./Data/trialIDVirt1.csv', row.names=F)

catchData <- df[c("SampleID","Species", "Returned.Weight", "Returned.Number", "Retained.Weight", 
           "Retained.Number")]

catchData <- setDF(setDT(catchData)[, lapply(.SD, sum, na.rm=TRUE),
                    by=c('SampleID', 'Species'), 
                    .SDcols=c("Returned.Weight", "Returned.Number", "Retained.Weight",  "Retained.Number")])

#I need to make a primary (unique) key to avoid uploading duplicates

catchData$CountID <- paste(catchData$SampleID, catchData$Species, sep='_')
catchData <- catchData[c("SampleID", "CountID", "Species", "Returned.Weight", "Returned.Number", "Retained.Weight",  "Retained.Number")] 
write.csv(catchData, file='./Data/catchDataVirt1.csv', row.names=F)
```

## Reading the track data from Followmee

This is probably the least painful component. All I need to do is read the saved csv from Followmee, and add some identifiers to the data set. Note in the case of Virtuous, I had to delete the section of Tom's car trip from Hull to Whitby.

```
#Change directory/filename as required
track <- read.csv('./Data/VirtuoustrackLastRecord.csv', stringsAsFactors=F)
track$ID <- "Virtuous"
track$keyID <- paste0(track$ID, track$Date)

# Delete the first part
track$DTime <- as_datetime(track$Date)
track <- subset(track, DTime > as_datetime('2021-04-13 04:00:00 PM'))
track$DTime <- NULL
write.csv(track, file='./Data/trackDataVirt1.csv',
          na = "",
          row.names=F)
```

## Reading the manually entered Effort data
At present I can't reliably identify trawl start and stop from Followmee. So, I manually examine the track and cop-paste the position and time from what I can see is the beginning end of the trawl. This is entered in a CSV file, and read in a written to another database table. Note I measure the trawl duration in both hours and minutes, although we will use hours for the CPUE correction:  

```
df <- read.csv('./Data/VirtuousEffort1.csv', stringsAsFactors=F)
df$SampleID <- paste(df$Asset, as.character(format(dmy(df$StartDate), "%d/%m/%Y")), df$Activity, sep='_')
df$TowTimeMinutes <- difftime(ymd_hms(df$TrawlEndTime), ymd_hms(df$TrawlStartTime), units="mins")
df$TowTimeHours <- difftime(ymd_hms(df$TrawlEndTime), ymd_hms(df$TrawlStartTime), units="hours")

df$tempdat <- as.character(format(dmy(df$StartDate), "%d/%m/%Y"))
df$tempdat

# MySQL reads year-month-day into dates
df$year <- substr(df$tempdat, 7, 10)
df$month <- substr(df$tempdat, 4, 5)
df$day <- substr(df$tempdat, 1, 2)

df$YearMonthDay <- paste(df$year, df$month, df$day, sep='-')

df$year <- NULL
df$month <- NULL
df$day <- NULL
df$tempdat <- NULL
```

For Virtuous and Golden Ray this value directly corresponds with the trialID key variable. Ideally we would need only the one effort table. However... we can't do this for Eilidh Anne because we only have the beginning and start of the day - not the haul. So for Eilidh Anne we have a separate table.  

## Renaming and loading the image filenames
This step is definitely one that you need to keep on top of. Typically there will be four hauls per day, so the number of photos quickly stacks up. Manually renaming the images is not a viable option. There is too much potential for silly typo errors.  
The approach I use is to read the unique SampleID variable from the trialID table, and directly parse the identifier into the filename. To do this, we need the images to be renamed to be in a separate directory. I get a listing of the filenames, then go through the WhatsApp messages to make sure the images are correctly lined with the SampleID variables.  
For the most part these are photos of the hopper. In some cases they are elsewhere (eg. the sample), so I need to add an identifier into the filename.  

```{note}
It is possible to store the image itself i a MySQL database as a BLOB (Binary Large Object) entry. However nobody in database land recommends it, so I simply save the filename in the database table to allow retrieval of the correct photo.
```

The simplified coding looks a bit like this...

```
frompath <- './2021_04_VirtuousPhotos/'
topath <- './2021_04_VirtuousPhotosRenamed/'
list.files('frompath')
trialID <- trialID$SampleID

inittable <- data.frame(SampleID=character(0), SampleImageFile=character(0))
inittable$SampleID <- as.character(inittable$SampleID)
inittable$SampleImageFile <- as.character(inittable$SampleImageFile)

rename <- function(FROM, OF, IDNUM){
    file1 <- paste0(frompath, FROM)
    id1 <- trialID[IDNUM, 'SampleID']
    file2 <- paste(id1, OF, sep='_')
    file2 <- paste(file2, 'JPG', sep='.')
    file2 <- gsub("[[:space:]]", "_", file2)
    file2 <- gsub("/", "_", file2)
    file2 <- paste0(topath, file2)
    !file.copy(from=file1, to=file2)
}

rename('IMG_0009.JPG', 'Hopper', 2)
# Do this for all the image files in the list

# Now write the new image names to a csv to be imported to the database
filelist <- data.frame(list.files('./2021_04_VirtuousPhotosRenamed/'))
names(filelist) <- 'file'
filelist$SampleID <- substr(filelist$file, 1, 25)
filelist$SampleID <- gsub("_", "/", filelist$SampleID)
filelist$SampleID <- gsub("s/", "s_", filelist$SampleID)
filelist$SampleID <- gsub("/H", "_H", filelist$SampleID)

filelist$PhotoLocation <- substr(filelist$file, 27, 32)

filelist <- filelist[c('SampleID', 'file', 'PhotoLocation')]
filelist
write.csv(filelist, file='./Data/deckPhotoFileVirt1.csv',
         row.names=F)
```

Pretty ugly, huh?